In [4]:
#AI-powered lane wise traffic counter built using YOLOv8 and Python.


In [5]:
# -*- coding: utf-8 -*-
"""traffic_flow_analysis.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1FXqpnSjjWl_su3GC5ZbuCxngFUE7V6T4#scrollTo=ht-eRGf252Nr
"""

# =========================
# Traffic Flow Analysis - Counting Lines (Colab-ready, fixed)
# =========================

# ---- SETTINGS ----
SOURCE_URL = "https://www.youtube.com/watch?v=MNn9qKG2UFI"  # change if needed
OUTPUT_VIDEO = "output_overlay.mp4"
OUTPUT_CSV = "vehicle_counts.csv"
MODEL_NAME = "yolov8n.pt"
COUNT_BOTH_DIRECTIONS = False  # False = only top->bottom crossing; True = both directions
MIN_CONF = 0.35

# ---- INSTALL DEPENDENCIES ----
!pip install ultralytics opencv-python yt-dlp numpy scipy --quiet

# ---- IMPORTS ----
import time, csv
from collections import defaultdict
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment
from ultralytics import YOLO
from google.colab import files

# ---- CONFIG ----
VEHICLE_CLASS_IDS = {2, 3, 5, 7}  # COCO: car=2, motorcycle=3, bus=5, truck=7
IOU_MATCH_THRESHOLD = 0.3
MAX_MISSES = 30

# ---- TRACKER (lightweight SORT-like) ----
def iou(bb1, bb2):
    xx1 = max(bb1[0], bb2[0]); yy1 = max(bb1[1], bb2[1])
    xx2 = min(bb1[2], bb2[2]); yy2 = min(bb1[3], bb2[3])
    w = max(0., xx2 - xx1); h = max(0., yy2 - yy1)
    inter = w * h
    a1 = (bb1[2]-bb1[0])*(bb1[3]-bb1[1]); a2 = (bb2[2]-bb2[0])*(bb2[3]-bb2[1])
    union = a1 + a2 - inter
    return inter/union if union>0 else 0.0

class Track:
    def __init__(self, bbox, tid):
        self.bbox = np.array(bbox, dtype=float)
        self.track_id = tid
        self.hits = 1
        self.misses = 0
        self.prev_centroid = None
    def update(self, bbox):
        self.bbox = np.array(bbox, dtype=float)
        self.hits += 1
        self.misses = 0
    def mark_missed(self):
        self.misses += 1
    def centroid(self):
        x1,y1,x2,y2 = self.bbox
        return np.array([(x1+x2)/2.0, (y1+y2)/2.0])

class SimpleTracker:
    def __init__(self, iou_threshold=IOU_MATCH_THRESHOLD, max_misses=MAX_MISSES):
        self.iou_threshold = iou_threshold
        self.max_misses = max_misses
        self.next_id = 1
        self.tracks = []
    def update(self, detections):
        if not self.tracks:
            for d in detections:
                self.tracks.append(Track(d, self.next_id)); self.next_id += 1
            return self.tracks
        N, M = len(self.tracks), len(detections)
        cost = np.ones((N, M))
        for i,tr in enumerate(self.tracks):
            for j,det in enumerate(detections):
                cost[i,j] = 1.0 - iou(tr.bbox, det)
        row_idx, col_idx = linear_sum_assignment(cost)
        assigned_t, assigned_d = set(), set()
        for r,c in zip(row_idx, col_idx):
            if r < N and c < M and (1.0 - cost[r,c]) >= self.iou_threshold:
                self.tracks[r].update(detections[c]); assigned_t.add(r); assigned_d.add(c)
        for i,tr in enumerate(self.tracks):
            if i not in assigned_t: tr.mark_missed()
        for j,det in enumerate(detections):
            if j not in assigned_d:
                self.tracks.append(Track(det, self.next_id)); self.next_id += 1
        # remove dead
        self.tracks = [t for t in self.tracks if t.misses <= self.max_misses]
        return self.tracks

# ---- Count-line manager (one line per lane + x-range for that lane) ----
class CountingLineManager:
    def __init__(self, lanes):
        # lanes: list of dicts with keys: line_x, line_y, x_min, x_max
        self.lanes = lanes
    @staticmethod
    def for_three_vertical_lanes(w, h, y_frac=0.62, shrink=10):
        lane_w = w // 3
        y_line = int(h * y_frac)
        lanes = []
        for i in range(3):
            left = max(0, i*lane_w + shrink)
            right = min(w, (i+1)*lane_w - shrink)
            cx = (left + right)//2
            lanes.append({'line_x': cx, 'line_y': y_line, 'x_min': left, 'x_max': right})
        return CountingLineManager(lanes)
    def draw(self, frame):
        for idx, ln in enumerate(self.lanes, start=1):
            x = int(ln['line_x']); y = int(ln['line_y'])
            cv2.line(frame, (x-40,y), (x+40,y), (0,0,255), 3)  # small horizontal segment centered on lane center
            cv2.putText(frame, f"Line {idx}", (x-30, y-12), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
            # optional: show lane bounds
            cv2.rectangle(frame, (ln['x_min'], 0), (ln['x_max'], frame.shape[0]), (0,255,255), 1)

# ---- YouTube download ----
def download_youtube(url, out_path="input_video.mp4"):
    import yt_dlp
    with yt_dlp.YoutubeDL({'format':'best[ext=mp4]/best','outtmpl':out_path,'quiet':True}) as ydl:
        print("Downloading video... (this can take a minute or two)")
        ydl.download([url])
    return out_path

# ---- Main processing ----
def process_video(source, out_video, out_csv, model_name):
    if source.startswith('http'):
        source = download_youtube(source)
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError("Unable to open video: "+str(source))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Video opened: {w}x{h} @ {fps:.2f} FPS")
    out_writer = cv2.VideoWriter(out_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w,h))
    model = YOLO(model_name)  # will auto-download if needed
    tracker = SimpleTracker()
    lines = CountingLineManager.for_three_vertical_lanes(w,h, y_frac=0.62, shrink=12)

    csv_f = open(out_csv, 'w', newline='')
    csv_w = csv.writer(csv_f); csv_w.writerow(['vehicle_id','lane_number','frame','timestamp_s'])

    counted = set()            # (track_id, lane_idx)
    per_lane_counts = defaultdict(int)

    frame_idx = 0
    t0 = time.time()
    print("Starting processing...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        # detect
        results = model(frame, imgsz=640, conf=MIN_CONF, verbose=False)
        detections = []
        for res in results:
            boxes = getattr(res, 'boxes', None)
            if boxes is None:
                continue
            for box in boxes:
                # box.cls, box.conf, box.xyxy
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                if cls in VEHICLE_CLASS_IDS and conf >= MIN_CONF:
                    x1,y1,x2,y2 = box.xyxy[0].cpu().numpy()
                    detections.append([float(x1), float(y1), float(x2), float(y2)])

        active_tracks = tracker.update(detections)

        # draw lines & lane visuals
        lines.draw(frame)

        # For each track -> determine lane by x coordinate, then check crossing of that lane's line_y
        for tr in active_tracks:
            cx, cy = tr.centroid()
            lane_found = None
            lane_idx = None
            for idx, ln in enumerate(lines.lanes, start=1):
                if cx >= ln['x_min'] and cx <= ln['x_max']:
                    lane_found = ln
                    lane_idx = idx
                    break

            # draw bbox + id
            x1,y1,x2,y2 = map(int, tr.bbox)
            color = (0,255,0) if (tr.track_id, lane_idx) in counted else (0,165,255)
            cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
            cv2.putText(frame, f"ID {tr.track_id}", (x1, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            if lane_found is not None:
                line_y = lane_found['line_y']
                prev = tr.prev_centroid
                if prev is not None:
                    prev_y = prev[1]
                    if not COUNT_BOTH_DIRECTIONS:
                        crossed = (prev_y < line_y <= cy)
                    else:
                        crossed = (prev_y < line_y <= cy) or (prev_y > line_y >= cy)
                    if crossed and (tr.track_id, lane_idx) not in counted:
                        counted.add((tr.track_id, lane_idx))
                        per_lane_counts[lane_idx] += 1
                        csv_w.writerow([tr.track_id, lane_idx, frame_idx, f"{frame_idx/fps:.3f}"])
                        # mark bounding box green
                        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
                # end prev check
            # end lane_found

            # update prev centroid AFTER processing all lanes for this track
            tr.prev_centroid = (cx, cy)

        # draw counts
        for i in range(1, 4):
            cv2.putText(frame, f"Lane {i}: {per_lane_counts[i]}", (10, 30 + 30*i),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

        out_writer.write(frame)

        # optional: print progress every 200 frames
        if frame_idx % 200 == 0:
            print(f"Processed {frame_idx} frames...")

    cap.release()
    out_writer.release()
    csv_f.close()
    print(f"Done: processed {frame_idx} frames in {time.time()-t0:.1f}s")
    print(f"Video saved: {out_video}")
    print(f"CSV saved: {out_csv}")

# ---- RUN ----
process_video(SOURCE_URL, OUTPUT_VIDEO, OUTPUT_CSV, MODEL_NAME)

# ---- SHOW & DOWNLOAD ----
from IPython.display import Video, display
display(Video(OUTPUT_VIDEO, embed=True))
print("Downloading files...")
files.download(OUTPUT_VIDEO)
files.download(OUTPUT_CSV)

Video opened: 640x360 @ 30.00 FPS
Starting processing...
Processed 200 frames...
Processed 400 frames...
Processed 600 frames...
Processed 800 frames...
Processed 1000 frames...
Processed 1200 frames...
Processed 1400 frames...


KeyboardInterrupt: 